In [29]:
import pandas as pd
import numpy as np

In [30]:
def transform_data(region, advancement=None, sex=None, age=None):
    loc_prot = '../../complete_data/' + region + '/' + region + '_Proteins.txt'
    loc_patient = '../../complete_data/PRIDEdataUploadInfo_231003.xlsx'
    df_prot = pd.read_csv(loc_prot, delimiter='\t')
    df_patient = pd.read_excel(loc_patient)

    df_patient = df_patient[df_patient["Brain region"] == region]


    # the following part is now implemented on js
            # # duplication of the patients between two advancements
            # advancements = ["C", "I", "II", "III", "IV", "V", "VI"]
            # for i in range(len(df_patient)):
            #     tmp = df_patient.iloc[i]["Braak stage"]
            #     if not tmp in advancements:
            #         row = pd.DataFrame([df_patient.iloc[i]])
            #         stages = tmp.split('-')
            #         df_patient.iloc[i, 4] = stages[0]
            #         row["Braak stage"] = stages[1]
            #         df_patient = pd.concat([df_patient, row], ignore_index=True)

            # if not advancement is None:
            #     df_patient = df_patient[df_patient["Braak stage"] == advancement]
            
            # if not sex is None:
            #     df_patient = df_patient[df_patient["Sex"] == sex]
            
            # if not age is None:
            #     df_patient["Age"] = pd.to_numeric(df_patient["Age"])
            #     df_patient = df_patient[df_patient["Age"] > age[0]]
            #     df_patient = df_patient[df_patient["Age"] < age[1]]


    # transforming the shape of the dataframe
    patients = df_patient["Sample code in PD search results"].to_list()
    patients_area = patients.copy()
    for i in range(len(patients_area)):
        patients_area[i] = "Area " + patients_area[i] + " Sample"
    df = df_prot[patients_area]
    df = df.T

    # renaming of the columns and indexes
    prots = df_prot["Description"].to_list()
    prots = [prot.split('OS=', 1)[0] for prot in prots]
    old_col_names = df.columns.to_list()
    dict_col = dict(zip(old_col_names, prots))
    dict_idx = dict(zip(patients_area, patients))
    df.rename(index=dict_idx, columns=dict_col, inplace=True)

    # adding the ages, sexes and advancements
    df["Age"] = df_patient[df_patient["Sample code in PD search results"].isin(patients)]["Age"].to_list()
    df["Sex"] = df_patient[df_patient["Sample code in PD search results"].isin(patients)]["Sex"].to_list()
    df["Braak stage"] = df_patient[df_patient["Sample code in PD search results"].isin(patients)]["Braak stage"].to_list()

    # writing to csv
    filepath = '../data/ProteinsPatients' + region + '.csv'
    df.to_csv(filepath)

In [31]:
transform_data("FC")
transform_data("TC")
transform_data("EC")
transform_data("PHC")
